In [1]:
import pandas as pd
import re
import sklearn

In [ ]:
pip install scikit-learn

In [14]:
df = pd.read_csv('output.csv')

In [15]:
df.columns

Index(['sentences'], dtype='object')

In [16]:
df

,sentences
0,"['국내외 경제동향 과 관련하여, 먼저 일부 위원은 올해 경제성장의 모멘텀 측면에서..."
1,"['국내외 경제동향 과 관련하여, 먼저 일부 위원은 부동산시장 정책방향과 관련하여 ..."
2,['국내외 경제동향 과 관련하여 일부 위원은 경제혁신 3개년 계획 등 정부의경제활성...
3,['제안설명에 이어 관련부서는 금번 연차보고서의 작성과 보완 등이 주관위원의 지도하...
4,['국내외 경제동향 과 관련하여 일부 위원은 국민계정의 새로운 국제기준2008 이행...
...,...
205,"['전일 개최된 동향보고회의에서는 다음과 같은 토의가 있었음.', '가 경제전망20..."
206,['제안설명에 이어 관련부서는 본 안건과 관련하여 위원협의회 등에서 논의된 내용을 ...
207,['이어서 관련부서는 9월 13일 위원협의회에서 논의된 주요 내용을 다음과 같이 보...
208,"['전일 개최된 동향보고회의에서는 다음과 같은 토의가 있었음.', '가 경제상황 평..."


### 사전필터

In [11]:
import pandas as pd
import os

# 불용어 리스트
stopwords = [
    "다른", "경우", "향후", "한편", "관련", "주요", "글로벌", "아울러", "따라", "통해", "에서", "있", "되", "한", "것", "원", "등", "년", "보", "억", "인", "수", "만", "적", "해", "중", "조", "다", "전", "대",
    "이날", "그", "낮", "내", "및", "경우", "개월", "개", "이번", "명", "점", "시", "데", "달", "화", "두", "미",
    "각각", "열", "연", "오", "엔", "성", "분", "총", "당", "후", "반면", "형", "계", "위", "약", "美", "하",
    "되", "년", "월", "자료", "있", "이", "한", "것", "적", "및", "일", "수", "보", "그림", "등", "억", "주",
    "원", "인", "들", "예상", "우", "대한", "당사", "별", "이후", "하나금융투자", "좌", "경우", "중", "해", "전",
    "미", "성", "인포맥스", "개", "하", "되", "있", "이", "등", "것", "한", "보", "적", "및", "수", "관련부서",
    "들", "인", "부", "최근", "대한", "대해", "점", "해", "경우", "중", "임", "때", "따라", "이후", "향후", "되",
    "있", "한", "보", "수", "말", "적", "인", "년물", "않", "나타나", "따르", "대한", "영향", "지난", "전", "이후",
    "만", "상황", "같", "전일", "올해", "시", "나오", "최근", "지표", "없", "더", "증", "기록", "이날", "현재",
    "위하", "다", "줄", "전망", "예상", "수준", "가능성", "거의", "어떤","웅진그룹","근근","코웨이","흥행작","세계","부문", 
    "궁색", "허황", "추호", "독무", "초본", "경복", "보라색", "꽃피", "별빛", "낙과", "문어", "숏장", "백운", "오감", "소청", "티레", "심리전", "영창", "고고", "홍성군", "화영", "가택", "기단", "골수", "궁중", "허츠", "휴게실", "선분", "설리", "국양제", "두건", "포렌식", "공갈", "귀여운", "동갑내기", "담요", "환갑", "백운", "긴급복지지", "직무급", "예스맨", "으쓱", "태운", "스나더", "도르", "아전", "피더", "화폐수량설", "명수", "열강", "빈축", "사토시", "굿윈", "에든버러", "마티스", "내거", "휴니드", "정치색", "카포", "숙취", "속편", "밥심", "라미", "토지념", "우분투", "어뢰", "뉴미디어", "푸싱", "닙의", "네스트", "반덤핑제소", "관광국", "빗댄", "웨보", "쉰러", "프론", "도망", "낙엽", "홈피", "피노", "다승", "리젠", "베개", "무치", "달라야", "숙의", "다와", "다호", "호구포역", "냉동고", "문판", "내부고발자", "부닥쳐", "긴다", "어처구니없", "고체연료", "안암", "슈워츠먼", "국위회", "진돗개", "박지성", "와세다", "오와주", "계속거래", "정동영", "예뻐", "직업인", "보까", "선라", "베네치아", "지푸라기", "습성", "명의자", "별빛", "난티", "외칠", "문규", "핵심국", "어벤져스", "차출", "유세장", "오쿤의법칙", "북부동맹", "투용지", "사운", "다우데타", "강영수", "4자회담", "장승배기역", "모니카", "완패", "늦장", "벽걸", "상임사국", "얼른", "화국", "태주", "노을", "호명", "의례", "웰메드", "바쿠", "쓸까", "라마스", "아시나", "보라매김", "선릉역", "폐회", "범식", "보관함", "무사고", "영창", "기뻐", "밤바", "목적인", "스쿠터", "계셨", "백현동", "오사무", "탄서", "검암역", "시나", "청호", "다윈", "고고", "홍성군", "아르테미스", "코스닥", "레딧", "양적완화", "통안채", "준비", "기회", "CD", "바오", "특수채", "CP", "파악", "RP", "능력", "불법", "역전폭", "단순", "외채", "취하", "흡수", "응답", "공간", "새마을금고", "게재", "스와프베시스", "숙박", "무렵", "요즘", "종금", "본점", "유출입", "과감", "만기별", "정밀", "차익거래", "커버드본드", "잠시", "트렌드", "발휘", "즉시", "별로", "정수단", "복현", "승진", "포착", "필요시", "환매조건부채권", "NAVER", "명절", "알파", "채권자금", "화채", "카지노", "보유비중", "생성", "벌어진", "신용거래", "답했", "기법", "독려", "시장조작", "생긴", "현명", "열심히", "시장안정화조치", "거둘", "가격제한폭", "코리보", "FF", "선택권", "채웠", "넉넉", "돌아왔", "롱숏", "오랜만", "순매수금액", "차분", "플레어", "막상", "악용", "변함없", "흔히", "ATM", "재점검", "본드스와프", "위함", "엄정", "매매주체", "셀트리온헬스케어", "외국자본", "느껴", "봐서", "스와프레트", "막연", "엿볼", "넓혔", "선물세트", "도우미", "순응", "보여야", "배당지수", "자금줄", "개별주식", "현혹", "갖출", "넓힐", "보유비율", "느낀다", "열광", "매입자산", "자문형랩", "매매전략", "딤섬본드", "찾아오", "사기범", "못한다고", "은퇴자금", "플라", "보유고", "포통장", "용한", "터주", "기밀", "스팸", "발품", "뛰어드", "도태", "거래주체", "김치본드", "권한다", "픽업", "뒤처지", "장난감", "찾아왔", "읶포맥스", "자금계획", "캐치", "토양", "고를", "나무가", "매수금액", "엄단", "허성우", "낯설", "최진호", "재빨리", "잠겨", "자만", "추주", "면역력", "교묘", "은닉", "소액채권", "농금채", "찍힌", "거금", "개구리", "회용품", "스마트머니", "김재홍", "순매수비중", "만끽", "풍족", "돌기", "스미싱", "FX마진거래", "증거금률", "보였었", "혜민", "주식보유비중", "돈벌", "미리미리", "나간다면", "문망", "용채", "애용", "롤링", "묶음", "서유", "화탁", "회용", "예감", "김태오", "함축", "엿보였", "풍수", "예외없", "유광열", "직감", "미수거래", "김예은", "irs", "은밀", "키워왔", "개별주식선물", "푼돈", "바꿔서", "섭취", "난다고", "취한다", "강구퉁", "부류", "풀어놓", "흘려", "만끽", "풍족", "돌기", "스미싱", "FX마진거래", "증거금률", "보였었", "혜민", "주식보유비중", "돈벌", "미리미리", "나간다면", "문망", "용채", "애용", "롤링", "묶음", "서유", "화탁", "회용", "예감", "김태오", "함축", "엿보였", "풍수", "예외없", "유광열", "직감", "미수거래", "김예은", "irs", "은밀", "키워왔", "개별주식선물", "푼돈", "바꿔서", "섭취", "난다고", "취한다", "강구퉁", "부류", "풀어놓", "흘려", "만기보유전략", "한미스프레드", "뿌려", "좌동", "채권보유비중", "북돋", "금발", "깨우", "찾아다니", "발할", "박종우", "나간다고", "써왔", "돌아볼", "재경보", "요긴", "뿌렸", "딤섬", "열심", "종채", "최창호", "시드머니", "신승진", "마음먹", "직매", "터줏감", "시스템트레딩", "유동성공급계획", "매수처", "냈었", "섺터", "곳저곳", "포미", "알고리즘트레딩", "터득", "메마른", "단둥", "환포지션", "오은", "KOSEF국고채10", "창현", "응했", "감귤", "화순", "메밀", "식과", "해했", "광엽", "국채선물", "채권시장", "순매수", "달러환율", "코스피", "장중", "순매도", "ECB", "Fed", "코스피지수", "뉴욕증시", "움직", "재료", "레벨", "돌파", "간밤", "저점", "낙폭", "삼성전자", "PCE", "하단", "IBK투자증권", "외환딜러", "IRS", "의사록", "미결제약정", "비농업", "머무르", "바닥", "근접", "네고물량", "우세", "최종호가", "레인지", "보합권", "미래에셋우", "소수의견", "국고3", "매도세", "다우존스30산업평균지수", "사였", "현물환", "OPEC", "밴드", "만장치", "부근", "테슬라", "탄탄", "선호심리", "수출업체", "뉴욕증권거래소", "드러나", "헤드라인", "다우지수", "횡보", "설문조사", "메시지", "추락", "사록", "막판", "민간평가사", "배당금", "플래트닝", "잔존", "내내", "스티프닝", "치를", "국채선물시장", "삼성선물", "코멘트", "결제수요", "깜짝", "연출", "연고점", "서베", "혼합형", "괜찮", "나흘", "저항선", "둬야", "테스트", "S&P500", "오퍼", "메타", "리포트", "블루칩", "지지력", "장후반", "본드웹", "브로커", "다우존스산업평균지수", "차신", "KEB하나은행", "느낌", "박승진", "두언", "돌아오", "네고", "김상훈", "피력", "딜링룸", "첫날", "장초", "NH선물", "줄곧", "터치", "근처", "탐색", "아웃퍼폼", "닥터", "연저점", "순매수세", "경계심리", "사퇴", "마리오드라기", "밤사", "재만", "플랫", "혼합", "동평균", "엠피", "롱스탑", "러드", "미선", "윤여", "트럭", "월초", "현선물", "점심", "나온다면", "리스토퍼", "환호", "어올", "목제", "수입차", "승지", "평탄", "반짝", "위쪽", "윤민", "경민", "나올지", "중간배당", "분봉", "공동락", "바킨", "순매도세", "어닝쇼", "돌아온", "아래쪽", "팽팽", "살짝", "편안", "쿠팡", "평선", "민경", "패트릭", "과태료", "재형", "ecb", "뒤집", "리콜", "베비스텝", "폭스바겐", "타겟", "뉴트럴", "개장후", "언더퍼폼", "중단기물", "분경", "한지영", "간다면", "CAC40", "KODEX200", "예쁜", "박옥희", "희종", "돌풍", "포진", "멘트", "반빅", "종목군", "질주", "잠깐", "나와야", "KOSPI200", "장단기물", "매입계획", "였으며", "특파", "외환선물", "깜빡", "볼까", "주저앉", "시초가", "강승", "생활형편", "위아래", "물러나", "우울", "삽입", "피겨", "매수강도", "갭업", "돌린", "현재생활형편", "낙폭과주", "KOSPI", "전광판", "박종연", "다가섰", "액면분할", "셀트리온제약", "굳건", "커졌", "김준", "트림", "롱스톱", "갇혀", "박민영", "턱밑", "갭다운", "마디", "솔솔", "박정우", "로고", "출렁였", "나와도", "버텨", "던진", "둔다", "준호", "머티", "스린", "박희찬", "기울었", "애셋", "갤런", "베어스티프닝", "빨간색", "파란색", "적색", "롱재료", "씨젠", "TIGER200", "놓였", "트럼프케어", "앞섰", "불플래트닝", "래피얼", "투자의견없음", "JPMorgan", "갈팡질팡", "신동준", "오르락내리락", "강해질", "아빅", "되돌림장세", "팽팽히", "빨간", "리터", "담뱃값", "발톱", "베어플래트닝", "혀질", "곤두박질", "힘겹", "속락", "급락장", "짓눌렀", "조영무", "드러낼", "형민", "파란불", "버겁", "순매도우위", "펼쳐졌", "전강후약", "성큼", "털썩", "버터", "어닝", "승환", "外人", "하락룸", "박성우", "나온다고", "픽업트럭", "점할", "삼전", "에치엘비", "세에", "저항력", "순매수우위", "맞해", "김구", "배민", "중공", "샤프", "나올까", "뒤따랐", "김학균", "숏재료", "나온다는", "김승혁", "곧어", "난주", "떠받쳤", "KODEX인버스", "뚫린", "바라봤", "코리아에셋투자증권", "유재혁", "커피숍", "불스티프닝", "훨훨", "당뇨", "봄바람", "아메", "느긋", "번갈", "김진평", "밀리지", "라코타", "성금요", "홀딩", "젠슨", "물러선", "빗나갔", "전해질", "줄까", "눌렀", "빙그레", "물러섰", "김다경", "선봉", "다가선", "덧붙임", "쌀쌀", "점했", "던져", "굳히", "바디", "간쑤", "남겨뒀", "느껴질", "피선", "김형렬", "마녀", "꿋꿋", "느껴진다", "로우볼", "마주옥", "본색", "승우", "달릴", "바로메드", "전약후강", "매도량", "매도강도", "민병규", "슬비", "곤두박질쳤", "김진욱", "서정훈", "햄버거", "신바람", "누를", "터보", "구랍", "박현", "함박웃음", "재탈환", "김승현", "갇히", "되돌아왔", "지켜질", "던질", "플린", "돌려놓", "약했", "뒷심", "구혜영", "블러드", "느껴졌", "Cl", "고승희", "구독료", "콜린", "네마녀의날", "김민규", "홍삼", "되돌아갈", "되돌아오", "근태", "슈나벨", "급전직하", "병열", "양봉", "허진욱", "김동욱", "황준호", "테윙", "우도", "음봉", "돌림", "헌터", "내줘", "읶니", "놀라울", "김밥", "온종", "박동진", "PCEPI", "굳어질", "눌릴", "남겨둘", "갈린", "갇혔", "되돌아갔", "네처셀", "눌린", "커머", "목균형", "박형중", "다가갈", "내릴까", "권리락", "매물벽", "필러", "졌었", "팔릴", "버틴", "갇힐", "키친", "네고우위", "머무른", "올란도", "쌍울", "러쉬", "날갯짓", "진호", "바겐", "킹덤", "박승영", "되밀리", "굳혔", "씽씽", "꽂힌", "볼린저밴드", "마틴루터킹데", "도넛", "갭상승", "자아냈", "플래트너", "볶음", "cac40", "피디아", "업종군", "와어", "굳힌다", "벽면", "코미팜", "김광현", "머문다면", "다다를", "울린", "갭하락", "스토캐스틱", "게걸음", "권희진", "드러냄", "굳혀", "프린터", "찍혔", "고꾸라질", "창민", "팝콘", "김홍식", "김두현", "던장", "헤드셋", "햇살", "오현", "KODEXBrazil", "그릴", "함량", "김지", "피파", "근창", "궐련형전자담배", "올려놨", "흘러간다면", "충만", "TIGER라틴", "굳어졌", "흐린", "쑤닝", "캐플란", "바싹", "노현우", "되돌아올", "갬블", "올라와서", "에릭슨", "주재료", "내보였", "ARIRANG고배당주", "받쳤", "JYPEnt", "리스피", "순매수량", "TIGER200IT레버리지", "찝찝", "두근두근", "창목", "선데토즈", "KODEX국고채", "움직까", "윤선정", "뒤쳐졌", "오르내림", "버라존", "곤충", "애썼", "콜럼", "받칠", "제친", "카야", "TIGER중국소비테마", "황민", "우치", "오밤", "굳힌", "근월", "옮겨졌", "시빅", "목욕탕", "핀터", "갈치", "홖윣", "키워갔", "김재중", "흘러나온다", "되돌아온", "였음에", "쿠첸", "그려진", "KODEXChinaH", "날아올랐", "파미셀", "핚은", "갈색", "김희웅", "엠마누엘", "소나타", "굳힐", "깜작", "최서영", "치약", "프린", "느낀다는", "바디텍메드", "신풍제약", "박재현", "애매모호", "전해진", "다우선물", "슬럼", "점심때", "오른", "박막", "임팔라", "탄산음료", "미끄러져", "5선", "미끄럼", "버틴다", "껌장", "움직였으며", "김경욱", "키퍼", "따돌렸", "삼총사", "머무름", "뛴다는", "황승택", "찔끔찔끔", "멈춘다", "키워나갔", "홀린", "KINDEX성장형F15", "바털", "와락", "TIGER본니케225", "울음", "생각", "라고", "모든", "너무", "빠지", "닷컴", "팬데믹", "강력", "확신", "디플레션", "간섭", "붕괴", "동력", "더상", "복귀", "그렇게", "행동", "승리", "BBB급", "뜨겁", "트레드", "버블", "트레딩", "차별", "의식", "무너", "단순히", "타밍", "컴퓨터", "페소", "라도", "음식", "과매도", "패닉", "발견", "중산층", "리라", "바람", "벌써", "복사", "가을", "지지선", "겨울", "극단적", "만든", "철저", "명품", "본질", "무조건", "탈출", "회귀", "아닌가", "역설", "영구", "편하", "눈앞", "재선", "버티", "키워드", "까닭", "오버슈팅", "될까", "믿음", "비관론", "진짜", "역풍", "용품", "세우", "손쉽", "펀더멘탈", "놀랍", "딜레마", "퇴출", "디플레", "힘든", "되풀", "버리", "놓치", "극우", "공황", "오해", "정본드", "처지", "탈환", "젊은층", "던지", "선인", "가볍", "불신", "나쁜", "엄청난", "좌파", "캐시", "TGIF", "각오", "파괴", "마리", "그야말로", "단숨에", "전락", "선보인", "뭉칫돈", "제프리", "테러준칙", "다우존스지수", "법칙", "설득력", "다먼", "조명", "희생", "강력매수", "한번", "비유", "초유", "근본", "로네", "농심", "귀결", "앞세워", "놀라", "메뉴", "차라리", "돌아올", "언젠가", "마지노선", "명백", "귀환", "탐욕", "열쇠", "김상", "특단", "농민", "균열", "장애물", "재앙", "버린", "꺼지", "히트", "못한다면", "옹호", "레시피", "단단", "분노", "가짜", "치킨", "부유", "양면", "면모", "낭비", "황제", "골드만", "띄우", "절박", "공짜", "똘똘", "싸우", "꽁꽁", "르펜", "헬리콥터머니", "머지않", "집착", "한우", "타완", "혼돈", "장바구니", "악몽", "관성", "억만장자", "납득", "지폐", "웬만", "안간힘", "엔고", "항공업", "자꾸", "두렵", "슈왑", "가슴", "드루", "포지셔닝", "떠올리", "아닐까", "돌아갔", "패리티", "끈적", "퍼펙트스톰", "파터", "쓰레기", "우월", "퇴진", "센서", "어선", "산다", "억울", "우유", "소나", "연상", "마인드", "돼지", "불패", "몰락", "사로잡", "블랙스완", "누군가", "노릴", "몰두", "급급", "갇힌", "단초", "후회", "좌절", "본다고", "가만히", "가죽", "에드워드", "버렸", "금세", "1월효과", "바라볼", "구멍", "했으니", "파도", "번번", "수수께끼", "쓰나미", "치부", "허베", "금물", "끈질기", "좌초", "역발상", "씀씀", "새삼", "통제력", "꺼낸", "다아몬드", "콜라", "피아", "앞세운", "반란", "사라", "처했", "매트", "불능", "징조", "막강", "러니", "어김없", "블랙홀", "평했", "틀렸", "별명", "용납", "드라", "떨치", "그럴", "칼날", "더더욱", "명백히", "낭패", "볼커룰", "빚내", "그건", "불렀", "반쪽", "닥친", "인사더", "유파동", "해야", "건재", "흥분", "조급", "태양", "불렸", "예외주의", "챙겼", "굴복", "아닌데", "고기", "엄두", "고군분투", "입히", "BB급", "수백만", "빚더미", "바통", "울리", "자초", "착각", "옛말", "김정현", "날렸", "결여", "단번에", "못지", "무르익", "오판", "달걀", "엔화값", "처하", "놓칠", "희미", "리우", "바깥", "닥칠", "작심", "그러면", "노랜", "작금", "리스탈", "으켜", "두둑", "통념", "볼커", "약탈", "돌아간", "휩쓸", "에브리", "받아여야", "끔찍", "불러으킬", "엄청", "역습", "역주행", "몰아넣", "퍼니", "언니", "불꽃", "엄밀히", "데니", "자르", "따라오", "농부", "리벳", "박탈감", "앞바다", "희생양", "마르코", "투영", "온다면", "로티", "바구니", "식탁", "조코", "닥치", "각인", "구경", "쌍둥", "쫓아가", "평온", "뜻밖", "좌시", "바라봐야", "내몰린", "렀다는", "초인플레션", "안겼", "막스", "올라타", "야금", "닥터둠", "황금알", "버릴", "총알", "합성어", "통증", "푸념", "낙하", "골몰", "마른", "따져볼", "슬슬", "좀체", "쐐기", "아킬레스건", "거스르", "끊겼", "배신", "온갖", "빼앗", "능사", "오롯", "울프", "애쓰", "노심초사", "마초", "착취", "카터", "머신", "레제프", "무서운", "온탕", "요란", "무덤", "불닭", "써서", "혹평", "창궐", "단백질", "내놔야", "상투", "끊긴", "궁지", "날까", "억지", "사치", "셀렉", "극우파", "경제붕괴", "새우", "목마른", "돌려야", "닥쳤", "갖춰졌", "와도", "우린", "혐오", "혈안", "조롱", "정공법", "스러움", "나락", "화끈", "따라야", "쳐다보", "퇴치", "금태환", "와야", "마세요", "체메트", "만병통치약", "병자", "깔렸", "솔깃", "가벼운", "탄탄히", "애써", "릭스미스", "끌어낼", "놔두", "되팔", "여겼", "라니", "카나리아", "식용유", "내버려", "초콜릿", "궁핍", "곤경", "낙인효과", "올지", "횡포", "삼겹살", "힘없", "볼멘", "디저트", "김상봉", "김준태", "거지", "어야", "얼핏", "단박", "건드리", "목도", "십상", "오븐", "질식", "견뎌", "열지수", "데자뷔", "몰아치", "제발", "싸울", "몰려오", "냉탕", "여겨져", "돈나무", "미숙", "스모그", "자경단", "음식물", "산다는", "강렬", "저절로", "멀쩡", "황금기", "악기", "쿠나", "몸통", "조바심", "황소", "탈선", "올까", "한탄", "올겨울", "과속", "제압", "부딪히", "침착", "여태", "경종", "당길", "보지않는손", "암시장", "설파", "생수", "종언", "탄약", "딱지", "비성적과열", "슬그머니", "굳히기", "식히", "키는", "채찍", "허약", "미증유", "버려야", "맹신", "포탈", "생리", "날려", "탄광", "단번", "날벼락", "빨래", "노태우", "괜히", "고압", "확언", "퇴짜", "닉슨쇼", "두려워", "오죽", "필사", "백약", "충동", "온다고", "우랄", "지우마", "린치", "레머", "펄펄", "어떻든", "뒤따라야", "펼쳐진", "잠적", "예주사", "미운", "붉은색", "갸우뚱", "모른다고", "치렀", "양자컴퓨터", "부유세", "투지", "씨름", "반지", "가로챈", "애덤", "식용", "막연히", "멘탈", "괴롭히", "까먹", "불행히", "코팅", "레벨2", "그래서인지", "돌아봤", "맹공", "치즈", "부딪힐", "야성적충동", "바주카포", "눈총", "초상", "민박", "살까", "빌려야", "허니", "전화위복", "태우", "발끈", "벼락", "바야흐로", "데빗", "여길", "멈칫", "슈퍼컴퓨터", "축포", "야채", "부딪치", "파팅", "제쳤", "끝판", "마구잡", "합쳐진", "그린스펀수수께끼", "안휘", "뒤쫓", "한바탕", "둔갑", "척추", "범하", "락펴락", "멸종", "허탈", "배짱", "타락", "응징", "괴담", "달랬", "쇠고기", "되살릴", "군중", "뿔난", "노려야", "쪽박", "타박", "매섭", "울렸", "아찔", "맹목", "한없", "뜬금없", "뭘까", "꼼짝", "핀토", "놓쳐", "프린시펄", "사놓", "저질렀", "맷집", "완강", "잉태", "망할", "탄식", "렀다고", "소용없", "올라탄", "앞세우", "생색내", "떠올라", "열중", "뜻로", "편식", "환호성", "어떡", "부딪혀", "퍼트", "만든다고", "망고", "복면", "거침", "찻잔", "읽혔", "춤추", "풍겼", "버려", "농약", "버텨야", "휩쓸리", "바움", "바나나", "포켓몬고", "찍혀", "헤어나", "뒤집히", "뻔히", "끼쳐", "악셀", "3고시", "페퍼컴퍼니", "매서운", "에르메스", "약효", "헛발질", "떠올렸", "바꿀지", "레먼드", "헤매", "쓰인", "뒤흔드", "돌킬", "자민족주의", "바랄", "여백", "식기", "포켓몬", "악질", "재탕", "못추", "티타늄", "브러더스", "최윤범", "명퇴", "지켜낼", "래머", "필패", "휩쓴", "폭풍우", "비올", "선다면", "야마모토", "볼록", "볼지", "허공", "여지없", "데릭", "찜찜", "구해야", "메탄", "좌고우면", "흔려", "쓰러지", "따박따박", "뒤처질", "엄벌", "생색", "떠썩", "타기팅", "먹잇감", "근린궁핍화정", "닝샤", "안유", "풍기", "번다", "두박", "온난", "와튼", "혈압", "땜질", "안긴", "닌텐도", "불쾌", "슈팅", "청바",
    "포켓몬", "게임", "카지노", "장난감", "퍼즐", "보드게임", "비디오게임", "모바일게임", "온라인게임", "롤플레잉", "포커", "카드게임", "체스", "바둑", "곰", "소", "돼지", "닭", "고양이", "개", "새", "물고기", "사슴", "토끼", "꿀벌", "까마귀", "강아지", "고슴도치", "여우", "늑대", "호랑이", "사자",
    "밥상", "곰탕", "콜라", "샴푸", "치약", "식기세척기", "백미", "족발", "설리", "레시피", "버섯", "초콜릿", "햄버거", "생수", "커피숍", "빵집", "삼겹살", "바나나", "올리브유", "식용유", "김밥", "도넛", "피자",
    "애환", "마음먹", "슬럼프", "멘붕", "애착", "행복", "슬그머니", "짜증", "불쾌", "귀여운", "웃음", "눈물", "공포", "두려움", "감격", "기쁨", "애정", "미소", "울음", "사랑",
    "꽃다발", "바람", "태양", "눈앞", "가을", "겨울", "바다", "강", "산", "숲", "꽃", "나무", "해", "달", "구름", "비", "눈", "바위", "강물", "호수",
    "유교", "신앙", "평화상", "문화", "전통", "역사", "궁중", "종교", "민속", "축제", "공연", "음악", "예술", "전래", "유물", "고미술품", "문어", "고전",
    "심장병", "결핵", "혈압", "안과", "치과", "병원", "약국", "의사", "간호사", "진료", "수술", "진단", "치료", "환자", "약물", "주사", "항생제", "백신", "건강검진",
    "배드민턴", "골프", "축구", "야구", "농구", "테니스", "수영", "조깅", "자전거", "운동", "체육", "스포츠", "경기", "선수", "팬", "응원", "팀", "트레이닝",
    "곰", "소", "돼지", "닭", "고양이", "개", "새", "물고기", "사슴", "토끼", "꿀벌", "까마귀", "강아지", "고슴도치", "여우", "늑대", "호랑이", "사자", "KT","한화생명","준다","LG","LG전자","SK텔레콤","한항공","기아","두산","삼성중공업","삼성화재","삼성SDI","LG디스플레","현모비스","CJ","GS건설",
    "현차투자증권","노인","유튜브","LG유플러스","우리"
]

# 파일 경로 (절대 경로 사용)
base_path = r"C:\Users\yoons\Desktop\금리예측포르젝트\test\병합할파일\FT\polarity_bootstrap"
hawkish_file = os.path.join(base_path, "제거_hawkish.csv")
dovish_file = os.path.join(base_path, "제거_dovish.csv")
hawkish_output_file = os.path.join(base_path, "2차hawkish_filtered.csv")
dovish_output_file = os.path.join(base_path, "2차dovish_filtered.csv")

# CSV 파일 읽기 및 불용어 제거
def filter_stopwords(input_file, output_file, column_name):
    try:
        # CSV 파일 읽기
        df = pd.read_csv(input_file)
    except FileNotFoundError:
        print(f"파일을 찾을 수 없습니다: {input_file}")
        return
    
    # 불용어 제거
    original_count = len(df)
    df_filtered = df[~df[column_name].isin(stopwords)]
    filtered_count = len(df_filtered)
    
    # 결과 저장
    df_filtered.to_csv(output_file, encoding='utf-8-sig', index=False)
    print(f"{input_file} 처리 완료: {original_count} → {filtered_count} 단어 (제거된 단어: {original_count - filtered_count})")

# hawkish.csv 처리
filter_stopwords(hawkish_file, hawkish_output_file, "우리")

# dovish.csv 처리
filter_stopwords(dovish_file, dovish_output_file, "우리")

C:\Users\yoons\Desktop\금리예측포르젝트\test\병합할파일\FT\polarity_bootstrap\제거_hawkish.csv 처리 완료: 1533 → 1533 단어 (제거된 단어: 0)
C:\Users\yoons\Desktop\금리예측포르젝트\test\병합할파일\FT\polarity_bootstrap\제거_dovish.csv 처리 완료: 1849 → 1849 단어 (제거된 단어: 0)


#### TF-IDF 테스트 셋

In [20]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from collections import Counter

# 🔹 감성 사전 단어 불러오기
dovish_words = pd.read_csv("dovish_filtered.csv", header=None)[0].tolist()
hawkish_words = pd.read_csv("hawkish_filtered.csv", header=None)[0].tolist()
all_words = list(set(dovish_words + hawkish_words))  # 중복 제거

# 🔹 정책 문서 문장 불러오기
df = pd.read_csv("output.csv")  
documents = df["sentences"].dropna().tolist()

# 🔹 단어 빈도 계산 (최소 등장 횟수 3 이상)
word_list = " ".join(documents).split()
word_counts = Counter(word_list)
filtered_words_freq = [word for word in all_words if word_counts[word] >= 1]

# 🔹 TF-IDF 적용 (IDF 사용 X)
vectorizer = TfidfVectorizer(vocabulary=filtered_words_freq, use_idf=True)
tfidf_matrix = vectorizer.fit_transform(documents)

# 🔹 단어별 평균 TF 점수 계산
tfidf_scores = np.asarray(tfidf_matrix.mean(axis=0)).flatten()
word_tfidf = dict(zip(filtered_words_freq, tfidf_scores))

# 🔹 중복 단어 제거
filtered_words = list(set(word_tfidf.keys()))  # 중복 단어 제거

# 🔹 TF-IDF 점수 0.03 이상인 단어만 남기기
filtered_words = [word for word in filtered_words if word_tfidf[word] > 0.05]

# 🔹 상위 3000개 단어 선택 (TF 점수 높은 순)
sorted_words = sorted(word_tfidf.items(), key=lambda x: x[1], reverse=True)[:3000]

# 🔹 비둘기파 / 매파 단어 다시 분류
filtered_dovish = [word for word, score in sorted_words if word in dovish_words]
filtered_hawkish = [word for word, score in sorted_words if word in hawkish_words]

# 🔹 결과 저장
pd.DataFrame(sorted_words, columns=["word", "tf"]).to_csv("tf_scores.csv", index=False)
pd.DataFrame(filtered_dovish).to_csv("제거_dovish.csv", index=False, header=False)
pd.DataFrame(filtered_hawkish).to_csv("제거_hawkish.csv", index=False, header=False)

print("✅ 필터링 완료!")
print(f"비둘기파 단어 개수: {len(filtered_dovish)}개")
print(f"매파 단어 개수: {len(filtered_hawkish)}개")


✅ 필터링 완료!
비둘기파 단어 개수: 2706개
매파 단어 개수: 2241개


In [7]:
# CSV 파일의 열을 확인
df = pd.read_csv('제거_hawkish.csv')
print(df.columns)


Index(['우리'], dtype='object')
